# SYSTEM ,  HUMAN AND AI MESSAGE TEMPLATE

In [2]:
from langchain_core.prompts import PromptTemplate , ChatPromptTemplate , SystemMessagePromptTemplate , HumanMessagePromptTemplate 

one_input_prompt=PromptTemplate.from_template("You are a helpful assitant that helps in {field}")
formatted_prompt=one_input_prompt.format(field="real estate")
response=groq_llm.invoke(formatted_prompt)
print("Response is " , response)

Response is  content="Hello! I'm thrilled to be your helpful assistant in the world of real estate. Whether you're a buyer, seller, or simply exploring your options, I'm here to provide guidance and support every step of the way.\n\nWhat can I help you with today? Are you:\n\n1. **Looking to buy a new home**? I can assist with finding properties that fit your criteria, understanding the home buying process, and connecting you with reputable real estate agents.\n2. **Selling your current home**? I can provide tips on staging, pricing, and marketing your property to attract potential buyers.\n3. **Researching neighborhoods**? I can give you insights into local communities, including information on schools, transportation, and amenities.\n4. **Exploring financing options**? I can help you understand mortgage rates, loan types, and other financial considerations.\n5. **Something else**? Feel free to share your question or concern, and I'll do my best to assist you.\n\nLet's get started!" r

In [8]:
sys_template = "You are a helpful {role_associate} assistant"
human_template= "{input}"

new_chat_prompt=ChatPromptTemplate.from_messages([

    SystemMessagePromptTemplate.from_template(sys_template),
    HumanMessagePromptTemplate.from_template(human_template)
])

print(new_chat_prompt)

formatted_chats=new_chat_prompt.format_messages(role_associate = "Real Estate" , input="I have 2 million dollars what will be the best iot in real estate and at which place in us")

groq_llm.invoke(formatted_chats)

input_variables=['input', 'role_associate'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['role_associate'], template='You are a helpful {role_associate} assistant')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))]


AIMessage(content="Congratulations on having $2 million to invest in real estate!\n\nWith $2 million, you have a wide range of investment opportunities in the US real estate market. Here are some of the best options to consider:\n\n**Option 1: Single-Family Rental Properties**\n\n* Location: Cities with strong rental demand and growing economies, such as:\n\t+ Austin, Texas\n\t+ Denver, Colorado\n\t+ Seattle, Washington\n\t+ Raleigh-Durham, North Carolina\n* Investment Strategy: Buy 2-3 single-family homes with a total value of $2 million, and rent them out to generate passive income.\n* Potential Returns: 8-12% annual cash flow, plus potential long-term appreciation.\n\n**Option 2: Apartment Building Investment**\n\n* Location: Urban areas with high demand for rental units, such as:\n\t+ Los Angeles, California\n\t+ New York City, New York\n\t+ Chicago, Illinois\n\t+ Boston, Massachusetts\n* Investment Strategy: Purchase a small to medium-sized apartment building (5-20 units) with a t

# OUTPUT PARSERS 


In [9]:
from langchain.output_parsers import PydanticOutputParser 
from langchain_core.pydantic_v1 import BaseModel , Field

class Crickter(BaseModel):
    name:str = Field(description="Crickter name")
    record : list = Field(description="The record that the crickter has made so far in a python list")

parser=PydanticOutputParser(pydantic_object=Crickter)

new_system_template="You are a hard core cricket fan and you have all the knowlegde about the cricket world"
human_template="{input} \n {formatted_output}"

chat_prompt_template=ChatPromptTemplate.from_messages([SystemMessagePromptTemplate.from_template(new_system_template) , HumanMessagePromptTemplate.from_template(human_template)])
formatting_template=chat_prompt_template.format_messages(input="Which is the best cricketer in the world?" , formatted_output=parser.get_format_instructions())


print(formatting_template)

response=groq_llm.invoke(formatting_template)
print("Response : "  ,response)


[SystemMessage(content='You are a hard core cricket fan and you have all the knowlegde about the cricket world'), HumanMessage(content='Which is the best cricketer in the world? \n The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"title": "Name", "description": "Crickter name", "type": "string"}, "record": {"title": "Record", "description": "The record that the crickter has made so far in a python list", "type": "array", "items": {}}}, "required": ["name", "record"]}\n```')]
Response :  content='What a question!\n\nHere is my answer:\n\n```\n{\n  "name": "Sir 

In [10]:
print("Formatted_response" , parser.parse(response.content))

Formatted_response name='Sir Don Bradman' record=['Highest career batting average of 99.94', '29 Test centuries', '13 half-centuries', '17,000+ First-class runs', '102.50 average in First-class cricket']


## Create React Agent Prompt Manually

In [11]:
from langchain.agents import create_react_agent , AgentExecutor
from langchain_core.prompts import PromptTemplate

template = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

react_prompt = PromptTemplate.from_template(template)



## Custom Tools Creation 

In [12]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain import hub
from langchain_core.prompts import PromptTemplate
from langchain.agents import create_react_agent, AgentExecutor
from langchain_groq import ChatGroq
from langchain_core.tools import ToolException
load_dotenv()
os.getenv("GROQ_API_KEY")
# Define the CalculatorInput schema
class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

def _handle_error(error: ToolException) -> str:
    return (
        "The following errors occurred during tool execution:"
        + error.args[0]
        + "Please try another tool."
    )

calculator = StructuredTool.from_function(
    func=multiply,
    name="Calculator",
    description="multiply numbers",
    args_schema=CalculatorInput,
    return_direct=True,
    handle_tool_error=_handle_error,

    # coroutine= ... <- you can specify an async method if desired as well
)

# Pull the prompt from the hub
prompt = hub.pull("hwchase17/react")

# Initialize the language model
groq_llm = ChatGroq(model_name="Llama3-70b-8192")

# Define the tools
tools = [calculator]

# Create the agent using the custom prompt
agent = create_react_agent(llm=groq_llm, prompt=prompt, tools=tools)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True , handle_parsing_errors=True)



# Example invocation
response = agent_executor.invoke({"input": "What is 5 times 3"})
print(response)



> Entering new AgentExecutor chain...
I need to multiply 5 and 3.

Action: Calculator
Action Input: "5*3"

ValidationError: 2 validation errors for CalculatorInput
a
  value is not a valid integer (type=type_error.integer)
b
  field required (type=value_error.missing)